<a href="https://colab.research.google.com/github/yasamanrazeghi7/BEtraining/blob/master/Distilling_Zero_Shot_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!git clone https://github.com/huggingface/transformers.git
!pip install transformers
!pip install datasets
!pip install torch

fatal: destination path 'transformers' already exists and is not an empty directory.


This notebook demonstrates how to use a script that provides a way to improve the speed and memory performance of a zero-shot classifier by training a more efficient student model from the zero-shot teacher's predictions over an unlabeled dataset.

For a given sequence, the zero-shot classification pipeline requires each possible label to be fed through the large NLI model separately. This requirement slows results considerably, particularly for tasks with a large number of classes K.

We'll use the AG's News topic classification dataset for this example.

In [24]:
from datasets import load_dataset
dev = load_dataset('mc_taco', split='validation')
dev[0]

Reusing dataset mc_taco (/root/.cache/huggingface/datasets/mc_taco/plain_text/1.1.0/8ebb9cb3e1a5bb32a5e9478ca6fda5d143d5a267ecde7b907d6b80e03d96e127)


{'answer': 'it sometimes was',
 'category': 4,
 'label': 0,
 'question': 'Is Islam still the majority religion?',
 'sentence': 'Islam later emerged as the majority religion during the centuries of Ottoman rule, though a significant Christian minority remained.'}

In [59]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()  # disable dropout for evaluation

# Encode a pair of sentences and make a prediction
tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.')
roberta.predict('mnli', tokens).argmax()  # 0: contradiction


Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


RuntimeError: ignored

### 🤗 Zero-shot classification pipeline

The [zero-shot classification pipeline](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.ZeroShotClassificationPipeline) is a tool withing 🤗 Transformers that can be used to classify text sequences out of the box, provided only a list of possible class names:

In [29]:
from transformers import pipeline
zero_shot_classifier = pipeline('zero-shot-classification', model="roberta-large-mnli", device=0)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
time_l = ['second', 'minute', 'hour', 'day', 'week', 'month', 'year', 'decade', 'century', 'centuries', 'times', 'never', 'daily', 'rarely', 'millions', 'frequently', 'every', 'once', 'twice']
def unique(list1):
  print(f'The original list was {len(list1)}')
  # insert the list to the set
  list_set = set(list1)
  # convert the set to the list
  unique_list = (list(list_set))
  print(f'Now changed to {len(unique_list)}')
  for x in unique_list:
    print(x)
ans_values = []
for i in range(len(dev)):
  if dev[i]['category'] ==  3:
    # print(dev[i])
    # input()
    ans_values.append(dev[i]['answer'])
    bool_flag = True
    # for tt in time_l:
    #   if tt in dev[i]['answer'] :
    #     bool_flag = False
    if bool_flag:
      print(dev[i]['answer'])

# unique(ans_values)
    


first sunday
later sunday
sunday
7th month
7th day
7:00 PM
it was morning
5:00 PM
7:00 AM
it was evening
3:00 PM
saturday
friday
8:00 PM
a second later
a hour later
7:00 PM
a minute before it started
7:00 AM
midnight
5:00 AM
at dawn
ratner first noticed a profit after expanding their market to the us, one hour ago on april 1st
ratner first noticed a profit after expanding their market to the us, one second ago on april 1st
2:00 PM
1:00 PM
it spend on the facility
about 12 am
about 3 am
about 1 am
about 2 am
detroit began talking about plastic-bodied cars
3:00 PM
3:00 AM
10:00 AM
5:00 PM
about 5 am
around 2 am
2:00 AM
about 10 am
12:00 PM
12:00 AM
after he left earlier
since he left earlier
tomorrow
before a couple days
the movie was released in the past hour
3:00 AM
12:00 AM
5:00 AM
9:00 PM
when he was a child
once he got to washington
1:00 AM
1:00 PM
1:00 AM
three seconds ago
the work gets the award
5:00 PM
1:00 PM
3:30 AM
midnight
3:30 PM
4:00 PM
2:00 AM
1:00 AM
noon
2:00 PM
6:00 PM


In [38]:
t_count = 0
class_names = ["true", "false"]
for i in range(len(dev)):
  a = zero_shot_classifier([dev[i]['sentence'] + " " + dev[i]['question'], dev[i]['answer']])
  print(a)
  ans = a['scores']
  if ans[0] > ans[1]:
    p_label = 1
  else:
    p_label = 0
  if p_label == dev[i]['label']:
    t_count = t_count + 1
print(t_count)



TypeError: ignored

In [ ]:
from transformers import pipeline
zero_shot_classifier = pipeline('zero-shot-classification', model="roberta-large-mnli", device=0)

In [ ]:
sequence = "A new moon has been discovered in Jupiter's orbit."
class_names = ["the world", "sports", "business", "science/tech"]
zero_shot_classifier(sequence, class_names)

{'labels': ['science/tech', 'the world', 'business', 'sports'],
 'scores': [0.7967991232872009,
  0.08699782937765121,
  0.07631051540374756,
  0.03989255428314209],
 'sequence': "A new moon has been discovered in Jupiter's orbit."}

This method serves as a convenient out-of-the-box classifier. Unfortunately, the method is by necessity somewhat slow. This is partially due to the large underlying model being used, but more important is the fact that for this method to work, every possible sequence / class name pair must be fed through the model together. So in order to classify `N` sequences into `K` classes, the model has to be called `N*K` times (whereas a typical classifier would only be called `N` times). This makes the method comparatively slow, especially for settings with a large number of classes.

In [ ]:
# classify 1600 examples with K=4 classes
%%time
for _ in range(100):
    zero_shot_classifier([sequence] * 16, class_names)

CPU times: user 6.23 s, sys: 2.83 s, total: 9.06 s
Wall time: 8.95 s


In [ ]:
# classify 1600 examples with K=8 classes
%%time
expanded_class_names = class_names + ["politics", "health", "food", "weather"]
for _ in range(100):
    zero_shot_classifier([sequence] * 16, expanded_class_names)

CPU times: user 10.6 s, sys: 6.96 s, total: 17.6 s
Wall time: 17.2 s


As we can see, increasing the number of classes from `K=4` to `K=8` approximately doubles the inference time. This classification method is extremely useful, but ideally we'd like to speed up inference.

### Distilling a more efficient student model

The best way to speed up inference is to **train a more efficient student model on the zero-shot classifier's predictions** over an unlabeled dataset. This can be done with the [`distill_classifier.py`](https://github.com/huggingface/transformers/blob/master/examples/research_projects/zero-shot-distillation/distill_classifier.py) script provided in the `transformers` repo.

Given (1) an unlabeled corpus and (2) a set of `K` class names, this script allows a user to train a standard classification head with `K` output dimensions. The script generates a softmax distribution for the provided data & class names, and a student classifier is then fine-tuned on these proxy labels. The resulting student model can be used for classifying novel text instances into the previously specified `K` classes with an order-of-magnitude boost in inference speed plus decreased memory usage.

Let's see how to do this with the [AG's News](https://huggingface.co/datasets/ag_news) topic classification dataset. The first thing we need is an unlabeled dataset (in reality AG's News is annotated of course, but we'll pretend and ignore the annotations for the sake of example). Let's put the sequences from the train set into a `txt` file:

In [ ]:
!mkdir agnews
with open("agnews/train_unlabeled.txt", 'w') as f:
    for seq in train["text"]:
        f.write(seq + '\n')

The other thing the script needs is the names of the classes. We'll put these into their own newline-delimitted `txt` as well:

In [ ]:
with open("agnews/class_names.txt", 'w') as f:
    for label in class_names:
        f.write(label + '\n')

Now we can run the script. First the zero-shot model will loop through the data and generate (soft) proxy-labels, and then a student `DistilBert` model will be fine-tuned on these predictions. The student will then be saved in `./distilbert-base-uncased-agnews-student`. See the [script readme](https://github.com/huggingface/transformers/blob/master/examples/research_projects/zero-shot-distillation/README.md) for more information about the available script arguments.

On a single P100, this will take about ~2 hours with the full training set of 130K examples. On a V100 with mixed precision (just pass `--fp16`), it will take ~30 minutes.

In [ ]:
!python transformers/examples/research_projects/zero-shot-distillation/distill_classifier.py \
--data_file ./agnews/train_unlabeled.txt \
--class_names_file ./agnews/class_names.txt \
--hypothesis_template "This text is about {}." \
--student_name_or_path distilbert-base-uncased \
--output_dir ./distilbert-base-uncased-agnews-student

2021-02-18 19:12:46.327937: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/18/2021 19:12:47 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
02/18/2021 19:12:47 - INFO - __main__ -   Training/evaluation parameters DistillTrainingArguments(output_dir='./agnews/distilled_model', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=128, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb

### Using the student model

The resulting model can now be loaded and used like any other pre-trained model:

(you can also use `"joeddav/distilbert-base-uncased-agnews-student"` to download this model from the hub if you want to try it without running whole script above)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./distilbert-base-uncased-agnews-student")
model = AutoModelForSequenceClassification.from_pretrained("./distilbert-base-uncased-agnews-student")
model.config

DistilBertConfig {
  "_name_or_path": "joeddav/distilbert-base-uncased-agnews-student",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "the world",
    "1": "sports",
    "2": "business",
    "3": "science/tech"
  },
  "initializer_range": 0.02,
  "label2id": {
    "business": 2,
    "science/tech": 3,
    "sports": 1,
    "the world": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.2",
  "vocab_size": 30522
}

and even used trivially with a `TextClassificationPipeline`:

In [ ]:
from transformers import TextClassificationPipeline
distilled_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
distilled_classifier(sequence)

[[{'label': 'the world', 'score': 0.1478172093629837},
  {'label': 'sports', 'score': 0.03247390314936638},
  {'label': 'business', 'score': 0.059586379677057266},
  {'label': 'science/tech', 'score': 0.7601224780082703}]]

Let's compare the speed & accuracy of the two methods.

Original zero-shot model:

In [ ]:
import numpy as np
from time import time
from tqdm.auto import tqdm

start = time()
batch_size = 32
hypothesis_template = "This text is about {}."
preds = []
for i in tqdm(range(0, len(test), batch_size)):
    examples = test[i:i+batch_size]['text']
    outputs = zero_shot_classifier(examples, class_names, hypothesis_template=hypothesis_template)
    preds += [class_names.index(o['labels'][0]) for o in outputs]
accuracy = np.mean(np.array(preds) == np.array(test['label']))
print(f"Teacher model accuracy: {accuracy*100:0.2f}%")
print(f"Runtime: {time() - start : 0.2f} seconds")


Zero-shot model accuracy: 69.33%
Runtime:  195.19 seconds


Distilled student model:

In [ ]:
start = time()
batch_size = 128 # larger batch size bc distilled model is more memory efficient
distilled_classifier.return_all_scores = False
preds = []
for i in tqdm(range(0, len(test), batch_size)):
    examples = test[i:i+batch_size]['text']
    outputs = distilled_classifier(examples)
    preds += [class_names.index(o['label']) for o in outputs]
accuracy = np.mean(np.array(preds) == np.array(test['label']))
print(f"Distilled model accuracy: {accuracy*100:0.2f}%")
print(f"Runtime: {time() - start : 0.2f} seconds")


Distilled model accuracy: 70.47%
Runtime:  11.27 seconds


As you can see, **the disitlled model gets similar accuracy on a held-out test set while running in 1/20th the time**. 

Lastly, you can share the distilled model with the community by [uploading it to the 🤗 Hub](https://huggingface.co/transformers/model_sharing.html). We've uploaded the distilled model from this notebook at [joeddav/distilbert-base-uncased-agnews-student](https://huggingface.co/joeddav/distilbert-base-uncased-agnews-student).